Notebook to collect the information of the generation

### Import

In [1]:
import sys, os, re, yaml, pickle
import commands
from glob import glob
sys.path.append('../lib')

import time, datetime

In [2]:
import numpy as np
from scipy.stats import mode
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import humanfriendly
from progressBar import ProgressBar

In [3]:
import uproot as ur
import ROOT as rt
rt.gErrorIgnoreLevel = rt.kError
rt.RooMsgService.instance().setGlobalKillBelow(rt.RooFit.ERROR)
import root_numpy as rtnp

Welcome to JupyROOT 6.12/07


# Inputs

In [4]:
sample_dic_dir = '/storage/user/ocerri/work/CMSSW_10_2_3/src/ntuplizer/BPH_RDntuplizer/production/samples.yml'

In [5]:
class Bauble: pass

In [6]:
setName = 'SignalRegion'
outDic = {}

outDic['mu'] = Bauble()
outDic['mu'].sample = 'B0_MuNuDmst_PU20'

outDic['tau'] = Bauble()
outDic['tau'].sample = 'B0_TauNuDmst_PU20'

outDic['Hc'] = Bauble()
outDic['Hc'].sample = 'B0_DmstHc_PU20'

outDic['Dstst'] = Bauble()
outDic['Dstst'].sample = 'Bp_MuNuDstst_PU20'

In [ ]:
# setName = 'SidePT'
# outDic = {}

# outDic['JPsiKst'] = Bauble()
# outDic['JPsiKst'].sample = 'B0_JpsiKst_PU0'

Gather the files

In [7]:
site_loc_conf = '/mnt/hadoop/store/user/ocerri'
ntuples_loc = '/storage/user/ocerri/BPhysics/data/cmsMC_private/{}/ntuples_B2*'
samples = yaml.load(open(sample_dic_dir))

for d in outDic.values():
    d.MINIAOD_dirs = []
    for directory in samples['samples'][d.sample]['parts']:
        aux = glob(site_loc_conf + directory[:-38].replace('ocerri-','') + '/*/*')
        d.MINIAOD_dirs += aux
    
    full_name = samples['samples'][d.sample]['dataset']
    d.ntuples_dir = glob(ntuples_loc.format(full_name))[0]

# Efficiency

In [8]:
def getEff(k,N):
    e = k/float(N)
    de = np.sqrt(e*(1-e)/N)
    return [e, de]

In [14]:
fn = glob(outDic['mu'].MINIAOD_dirs[-1]+'/*.root')[0]
print fn

/mnt/hadoop/store/user/ocerri/cmsMC_private_PU20_10-2-3/BPH_Tag-B0_MuNuDmst-pD0bar-kp_13TeV-pythia8_Hardbbbar_PTFilter5_0p0-evtgen_ISGW2_200117/200118_024956/0010/out_MINIAODSIM_10000.root


In [55]:
# load FWLite C++ libraries
rt.gSystem.Load("libFWCoreFWLite.so");
rt.gSystem.Load("libDataFormatsFWLite.so");
rt.FWLiteEnabler.enable()

In [58]:
# load FWlite python libraries
from DataFormats.FWLite import Lumis
from DataFormats.FWLite import Handle

In [69]:
handle = {}
handle['genInfo'] = [Handle('GenFilterInfo'), ('genFilterEfficiencyProducer', '', 'SIM')]

for lumi in Lumis(fn):
    prods = {}
    for k,v in handle.iteritems():
        lumi.getByLabel(v[1], v[0])
        prods[k] = v[0].product()
    print prods['genInfo'].numEventsPassed()
    print prods['genInfo'].numEventsTotal()
    print prods['genInfo'].filterEfficiency()
    print prods['genInfo'].filterEfficiencyError()

1099
150000
0.00732666666667
0.000220196679088


In [ ]:
N_max = 10000

for name, d in outDic.iteritems():
    print '\n--> ' + name
    dicOut = {}
    N_gen = 0
    N_cuts = 0
    xsec = []
    xsec_err = []
    
    print 'Fetching logs files location'
    step1log_list = []
    for directory in d.MINIAOD_dirs:
        step1log_list += glob(directory + '/step1log_*.root')
    if N_max > 0 and N_max < len(step1log_list):
        step1log_list = np.random.choice(step1log_list, N_max)
    
    print 'Analizing', len(step1log_list), 'logs'
    pb = ProgressBar(maxEntry=len(step1log_list))
    for i_j, step1log_file in enumerate(step1log_list):
#         print step1log_file
        pb.show(i_j)
        cmd = 'tail -n 40 ' + step1log_file
        status, step1_log_lines = commands.getstatusoutput(cmd)
        for il, line in enumerate(step1_log_lines.split('\n')):
            if line.startswith('Before Filter: total cross section'):
                aux = line[37:-3].split(' +- ')
                xsec.append(float(aux[0]))
                xsec_err.append(float(aux[1]))
            elif line.startswith('Filter efficiency (event-level)='):
                l=line[34:-24]
                l = l.split(') = ')[0]
                l = l.split(') / (')
                N_cuts += int(l[0])
                N_gen += int(l[1])
                break
        e, de = getEff(N_cuts, N_gen)
    
    xsec = np.array(xsec)
    xsec_err = np.array(xsec_err)
    s2 = np.square(xsec_err)
    num = np.sum(xsec/s2)
    den = np.sum(1./s2)
    xsec = 1e3*num/den
    xsec_err = 1e3*np.sqrt(1/den)
    print 'Xsec: {:1.4e} +/- {:1.4e} fb ({:1.1e})'.format(xsec, xsec_err, xsec_err/xsec)
    dicOut['xsec'] = [xsec, xsec_err]
    
    e, de = getEff(N_cuts, N_gen)
    print 'CMSSW eff: {:1.3e} +/- {:1.3e} ({:1.1e})'.format(e,de, de/e)
    dicOut['CMSSWFilterEff'] = [e, de]
    continue
    
    cand_out_list = glob(d.ntuples_dir + '/out/job*.out')
    N_analyzed = []
    N_trg = []
    N_cand = []
    eff = []
    for cand_out in cand_out_list:
        step5_log_lines = open(cand_out).readlines()
        eff_ln = []
        for line in reversed(step5_log_lines):
            if 'efficiency:' in line:
                eff_ln.append(line)

        aux = re.search('[0-9]+/[0-9]+', eff_ln[1]).group(0)
        aux = aux.split('/')
        N_analyzed.append(int(aux[1]))
        N_trg.append(int(aux[0]))
        
        aux = re.search(': [0-9]+/', eff_ln[0]).group(0)
        N_cand.append(int(aux[2:-1]))
    
    d.N_analyzed = np.array(N_analyzed)
    d.N_trg = np.array(N_trg)
#     print 'Avg N_trg:', float(np.sum(N_trg) / d.N_acc.shape[0])
    d.N_cand = np.array(N_cand)
#     print 'Avg N_cand:', float(np.sum(N_cand) / d.N_acc.shape[0])
    d.ntuplizerEff = d.N_cand.astype(np.float)/d.N_analyzed
    print 'Avg ntuplizer eff: {:1.2e}'.format(np.mean(d.ntuplizerEff))
    
    
    ## Sanity checks
    print 'Tot candidates:', np.sum(d.N_cand)
#     n1 = np.sum(d.N_analyzed)
#     n2 = np.sum(d.N_acc)
#     if n1 != n2:
#         print  np.sum(d.N_analyzed), np.sum(d.N_acc)
#         print n1-n2

    ## Save infos
    dump_d = {}
    dump_d['xsec'] = np.mean(d.xsec)
    dump_d['CMSSWFilterEff'] = np.mean(d.CMSSWFilterEff)
    dump_d['ntupplizerEff'] = np.mean(d.ntuplizerEff)   
    dump_d['TotCand'] = np.sum(d.N_cand)
    d.dump_d = dump_d

In [ ]:
dump_dic = {}
for n, d in outDic.iteritems():
    dump_dic[n] = d.dump_d
pickle.dump(dump_dic, open('../data/'+setName+'MC_efficiencies_PU20.pickle', 'wb'))

In [ ]:
for name, d in outDic.iteritems():
    print name
    fig, ax = plt.subplots(1, 3, figsize=(12,4))
    ax[0].hist(d.N_gen_cuts)
    ax[0].set_xlabel('Events after gen cuts')
    ax[1].hist(d.N_acc)
    ax[1].set_xlabel('Events accepted')
    ax[2].hist(d.N_cand)
    ax[2].set_xlabel('Candidates')

# Time

In [ ]:
def bashDate2Unix(ln, debug=False):
    template = '%c'
    ln = ln.replace('CEST ', '')
    ln = ln.replace('UTC ', '')
    if debug:
        print ln
    else:
        t = time.mktime(datetime.datetime.strptime(ln, template).timetuple())
        return float(t)

In [ ]:
for name in outDic.keys():
    print '\n--> ' + name
    master_dir = outDic[name].dir + '/'
    d = outDic[name]
    d.driver_time = []
    d.run_time =[]
    d.tot_time =[]
    
    job_dir_list = glob(master_dir + 'jobs_out/out_*[0-9]')
    pb = ProgressBar(maxEntry=len(job_dir_list))
    for i_j, job_dir in enumerate(job_dir_list):
        pb.show(i_j)
        aux = os.path.basename(job_dir)
        n_job = int(aux[4:])
        
        fn = glob(master_dir + 'jobs_out/out/job_{}_*.out'.format(n_job-1))[0]
        if not os.path.isfile(fn): continue
        test_log_lines = open(fn).readlines()
        
        try:
            crono = [[],[],[],[],[]]
            for i, ln in enumerate(test_log_lines):
                for j in range(1,5):
                    if 'Step '+str(j) in ln or 'Running step '+str(j) in ln:
                        try:
                            crono[j-1].append(bashDate2Unix(test_log_lines[i+1][:-1]))
                        except:
                            print '\nError in job', n_job
                            crono[j-1].append(crono[j-1][-1])
            crono[4].append(bashDate2Unix(test_log_lines[-1][:-1]))

            driver_time = []
            run_time = []
            for i in range(4):
                driver_time.append(crono[i][1] - crono[i][0])
                run_time.append(crono[i+1][0] - crono[i][1])
            tot_time = crono[4][0] - crono[0][0]

            d.driver_time.append(driver_time)
            d.run_time.append(run_time)
            d.tot_time.append(tot_time)
        except:
            pass
        
    d.driver_time = np.array(d.driver_time)
    d.run_time = np.array(d.run_time)
    d.tot_time = np.array(d.tot_time)

In [ ]:
bins = plt.hist(d.tot_time/3600)
plt.xlabel('Total time [h]')
x = np.percentile(d.tot_time, 90)/3600
plt.plot([x,x], [0,np.max(bins[0])], 'm--', lw=3);

# Disk

In [ ]:
for name in outDic.keys():
    print '\n--> ' + name
    master_dir = outDic[name].dir + '/'
    d = outDic[name]
    d.MINIAOD_size = []
    
    job_dir_list = glob(master_dir + 'jobs_out/out_*[0-9]')
    pb = ProgressBar(maxEntry=len(job_dir_list))
    for i_j, job_dir in enumerate(job_dir_list):
        pb.show(i_j)
        aux = os.path.basename(job_dir)
        n_job = int(aux[4:])
        if not os.path.isfile(job_dir + '/step1.log'): continue
        fname_MINI = master_dir + 'jobs_out/out_MINIAODSIM_{}.root'.format(n_job)
        if not os.path.isfile(fname_MINI): continue
        d.MINIAOD_size.append(os.path.getsize(fname_MINI))
    
    d.MINIAOD_size = np.array(d.MINIAOD_size)
    print 'Avg MINIAOD size:', humanfriendly.format_size(np.mean(d.MINIAOD_size), binary=True)

# Print

In [ ]:
for n, d in outDic.iteritems():
    table = PrettyTable()
    table.field_names = ['Step', 'Sub', 'Eff [%]', 'Driver time [s]', 'Running time/evt [s]']
    
    e, de = getEff(np.sum(d.N_acc), np.sum(d.N_gen_cuts))
    dt = np.mean(d.driver_time[:, 0])
    dr_ev = np.mean(d.run_time[:, 0]/d.N_acc)
    table.add_row(['GEN-SIM', '-', '{:.2f} +/- {:.2f}'.format(100*e, 100*de), '{:.1f}'.format(dt), '{:.1f}'.format(dr_ev)])
    
    e, de = getEff(np.sum(d.N_gen_cuts), np.sum(d.N_gen))
    table.add_row(['', 'Pythia Gen', 'x {:.1f}'.format(1./e), '-', '-'])
    table.add_row(['', 'Gen Filter', '100.0', '-', '-'])
    e, de = getEff(np.sum(d.N_acc), np.sum(d.N_gen_cuts))
    table.add_row(['', 'CMSSW Filter', '{:.2f} +/- {:.2f}'.format(100*e, 100*de), '-', '-'])

    table.add_row(['RAW', '-', '100.0', '{:.1f}'.format(np.mean(d.driver_time[:,1])), '{:.1f}'.format(np.mean(d.run_time[:,1]/d.N_acc))])
    table.add_row(['AOD', '-', '100.0', '{:.1f}'.format(np.mean(d.driver_time[:,2])), '{:.1f}'.format(np.mean(d.run_time[:,2]/d.N_acc))])
    table.add_row(['MINIAOD', '-', '100.0', '{:.1f}'.format(np.mean(d.driver_time[:,3])), '{:.1f}'.format(np.mean(d.run_time[:,3]/d.N_acc))])
    
    e, de = getEff(np.sum(d.N_cand), np.sum(d.N_acc))
    table.add_row(['CAND', '-', '{:.2f} +/- {:.2f}'.format(100*e, 100*de), '-', '-'])
    e, de = getEff(np.sum(d.N_trg), np.sum(d.N_acc))
    table.add_row(['', 'BPH Trg', '{:.2f} +/- {:.2f}'.format(100*e, 100*de), '-', '-'])
    e, de = getEff(np.sum(d.N_cand), np.sum(d.N_trg))
    table.add_row(['', 'Cand. sel.', '{:.2f} +/- {:.2f}'.format(100*e, 100*de), '-', '-'])
    
    table.add_row(len(table.field_names)*[''])
    
    e, de = getEff(np.sum(d.N_cand), np.sum(d.N_gen_cuts))
    table.add_row(['Tot', '', '{:.3f} +/- {:.3f}'.format(100*e, 100*de), '{:.1f}'.format(np.mean(np.sum(d.driver_time, axis=1))), '{:.1f}(*)'.format(np.mean(np.sum(d.run_time, axis=1)/d.N_acc))])

    print n
    print table.get_string()
    
    tabsum = PrettyTable()
    tabsum.field_names = ['Evts req.', '# MINIAOD Evts', '# Cand.', 'Tot. Time [h]']
    
    MINIAOD_size = humanfriendly.format_size(np.mean(d.MINIAOD_size), binary=True)
    tabsum.add_row(['{:.0f}k'.format(1e-3*np.mean(d.N_gen_cuts)), '{:.0f} ({})'.format(np.mean(d.N_acc), MINIAOD_size), '{:.0f}'.format(np.sum(d.N_cand) / d.N_acc.shape[0]), '{:.1f}'.format(np.mean(d.tot_time)/3600.)])
    print tabsum.get_string()
    